In [5]:
import numpy as np
import sys
import matplotlib.pyplot as plt
%matplotlib inline

In [108]:
import sys
from PIL import Image
import re
import glob
import random

def joinImages(imageFiles):
    imageList = []
    for i,imagePath in enumerate(imageFiles):
        image = Image.open(imagePath)
        if i == 0:
            leftSpacing = 20
            rightSpacing = 30
        elif i == len(imageFiles)-1:
            leftSpacing = 30
            rightSpacing = 20
        else:
            leftSpacing = 30
            rightSpacing = 30
            
        verticalRemoval = 5
        image = image.crop((leftSpacing, verticalRemoval, 128-rightSpacing, 128-verticalRemoval))
        imageList.append(image)
    
    width = 0
    height = imageList[0].size[1]
    for image in imageList:
        width += image.size[0]
    
    new_im = Image.new('RGB', (width,height))
    offset = 0
    for image in imageList:
        new_im.paste(image, (offset,0))
        offset+=image.size[0]
    return new_im

def getFontLabelsDict(filename):
    f = open('../font_labels.txt','rb')
    lines = f.readlines()
    f.close()
    d = {}
    for line in lines:
        line = re.sub(r'[\n\r]+', '', line)
        font_name, label = line.split(' ')
        d[font_name] = label
    return d

def getCharsForFont(fontName, path, fontLabels):
    result = {}
    for font in fontLabels:
        result[font] = {'digits':{},'lowercase':{}, 'uppercase':{}}

    for filename in glob.glob(path+'*.png'): #assuming png
        # Example file name ../new_color/ae_Jet--q.png
        name = filename.split('--')[0].split('/')[-1]
        c = filename.split('--')[-1].split('.')[0]        
        if ord(c) - ord('0') >=0 and ord(c) - ord('0') <=9:
            result[name]['digits'][c] = filename
        elif ord(c) - ord('a') >=0 and ord(c) - ord('a') <=25:
            result[name]['lowercase'][c] = filename
        else:
            result[name]['uppercase'][c] = filename
    return result

def getWord(fontDataDict):
    word_type = random.randint(0,3)
    word = ""
    if word_type == 0:
        # All Uppercase
        charList = fontDataDict['uppercase'].keys()
        word_length = random.randint(2,5)
        for i in xrange(word_length):
            word += charList[random.randint(0,len(charList)-1)]
        return word
    elif word_type == 1:
        # First Char Uppercase
        upperCharList = fontDataDict['uppercase'].keys()
        lowerCharList = fontDataDict['lowercase'].keys()
        word_length = random.randint(2,5)
        
        word += upperCharList[random.randint(0,len(upperCharList)-1)]
        for i in xrange(1, word_length):
            word = word + lowerCharList[random.randint(0,len(lowerCharList)-1)]
        return word
    elif word_type == 2:
        # All Lowercase
        charList = fontDataDict['lowercase'].keys()
        word_length = random.randint(2,5)
        for i in xrange(word_length):
            word += charList[random.randint(0,len(charList)-1)]
        return word
    else:
        # All Digits
        charList = fontDataDict['digits'].keys()
        word_length = random.randint(2,3)
        for i in xrange(word_length):
            word += charList[random.randint(0,len(charList)-1)]
        return word

In [91]:
fontLabels = getFontLabelsDict('../font_labels.txt')
result = getCharsForFont('Calligraph810-BT', '../letters-data/new_color/', fontLabels)

In [92]:
import pickle
pickle.dump( result, open( "font_image_dictionary.pickle", "wb" ) )

In [93]:
data = pickle.load( open( "font_image_dictionary.pickle", "rb" ) )
print len(data)

1866


In [109]:
count  = 0
for font, font_images in data.items():
    count+=1
    if count==10:
        fontDataDict = data[font]
        for j in xrange(30):
            word = getWord(fontDataDict)
            imageList = []
            for c in word:
                if ord(c) - ord('0') >=0 and ord(c) - ord('0') <=9:
                    imageList.append(fontDataDict['digits'][c])
                elif ord(c) - ord('a') >=0 and ord(c) - ord('a') <=25:
                    imageList.append(fontDataDict['lowercase'][c])
                else:
                    imageList.append(fontDataDict['uppercase'][c])
                image = joinImages(imageList)
                image.save(font+str(j)+'.png')